# Import libraries

In [25]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime
import extruct
from w3lib.html import get_base_url
import pprint

# Get title name&url for 2 pages (just for example)

In [26]:
def scrape_url_for_pages(url, p, title_name, title_url):
    res = requests.get(url+str(p))
    soup = BeautifulSoup(res.text,'html.parser')
    
    for description in soup.select('a.introa_medium.introa_news'):
        title_url.append(description['href'])
    
    for description in soup.select('a.introa_medium.introa_news > article > div.introa_rm_text > h2.introa_title'):
        title_name.append(description.text)
    
    return title_name, title_url

In [27]:
# You can choose to save urls to a file or just to dataframe
def scrape_urls_to_df(pages):
    title_name=[]
    title_url=[]
    
    url = "https://www.notebookcheck.net/?&hide_youtube=1&page="
    title_name, title_url = scrape_url_for_pages(url, pages, title_name, title_url)
    
    result = pd.DataFrame({'TITLE':title_name, 'TITLE_URL': title_url})
    #writer = pd.ExcelWriter(r'Scrap_notebookcheck_'+datetime.now().strftime('%Y-%m-%d')+'.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
    #result.to_excel(writer)
    #writer.close()
    return result

In [28]:
df = scrape_urls_to_df(2)

In [29]:
df.head()

,TITLE,TITLE_URL
0,Dell Prime Day slashes $200 off the XPS 15 957...,https://www.notebookcheck.net/Dell-Prime-Day-s...
1,"Windows Calculator ports for Android, iOS and ...",https://www.notebookcheck.net/Windows-Calculat...
2,Sharkoon claims its Gaming DAC Pro S can deliv...,https://www.notebookcheck.net/Sharkoon-claims-...
3,Motorola cancels plans to release Pie for its ...,https://www.notebookcheck.net/Motorola-cancels...
4,There are two Snapdragon 855-powered Samsung G...,https://www.notebookcheck.net/There-are-two-Sn...


# Use Pages in Step 1 to get articles

- In this function, we can get contents by single url

In [4]:
def scrap_news(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text,'html.parser')
    
    abstract = soup.select('div.news-abstract')[0].text
    author_date = soup.select('div.news-author-date')[0].text
    
    contents = []
    for t in soup.select('p.bodytext'):
        contents.append(t.text.strip())
    
    contents = " ".join(contents)
        
    return abstract, author_date, contents

- example

In [38]:
scrap_news(df['TITLE_URL'][0])

('Starting July 1st, Dell will be rolling out deals for its Inspiron, XPS, Alienware, Vostro, Precision, and G Series of PCs to piggy back on the success of Amazon Prime Day. Many of the price cuts are for older models in order to make room for the new models revealed at Computex 2019.',
 'by Allen Ngo, 2019/06/27',
 'Called "Black Friday in July", Dell will be joining other major retailers next month with deals and price cuts across a wide range of devices spanning personal PCs, gaming PCs, business PCs, and plenty of accessories as summarized below. It\'s important to note that the deals will not be in effect until July 1st and that many will be spread across a two-week period to minimize the risk of traffic overload and empty shopping carts at checkout. Of the major deals listed, notable ones include the XPS 15 9570 with Core i7-8750H for $1350 and the Alienware m15 R1 with Core i7-9750H for $1700. These systems will be refreshed very soon with the upcoming XPS 15 7590 and Alienware

# But actually, we want all the contents of urls:

In [59]:
def scrap_news_urls(url_list):
    abstract = []
    author_date = []
    article = []
    for url in url_list:
        
        res = requests.get(url)
        soup = BeautifulSoup(res.text,'html.parser')

        abstract.append(soup.select('div.news-abstract')[0].text)
        author_date.append(soup.select('div.news-author-date')[0].text)
        # if you want to split author and date
        # author = soup.select('div.news-author-date')[0].text.split(',')[0]
        # date = soup.select('div.news-author-date')[0].text.split(',')[1]

        contents = [] # collect all the contents (bodytext)
        for t in soup.select('p.bodytext'):
            contents.append(t.text.strip())

        contents = " ".join(contents)
        
        article.append(contents)
        
    return abstract, author_date, article

In [61]:
abstract, author_date, article = scrap_news_urls(df['TITLE_URL'])
result = pd.DataFrame({'TITLE':df['TITLE'], 'TITLE_URL': df['TITLE_URL'], 'ABSTRACT':abstract,
                      'AUTHOR_DATE': author_date, 'CONTENTS': article})

In [64]:
result.tail(3)

,ABSTRACT,AUTHOR_DATE,CONTENTS,TITLE,TITLE_URL
40,"Last year, OnePlus announced that it was to re...","by Deirdre O Donnell, 2019/06/25",OnePlus is mainly known as a maker of competit...,The OnePlus TV may not be OLED,https://www.notebookcheck.net/The-OnePlus-TV-m...
41,Samsung has allegedly filed a patent for a fol...,"by Deirdre O Donnell, 2019/06/25",Samsung could be said to be running out of tim...,Next-gen Galaxy Folds could have punch-hole ca...,https://www.notebookcheck.net/Next-gen-Galaxy-...
42,"At 67 C when gaming, you better make sure the ...","by Allen Ngo, 2019/06/25",We already noted in our review of the Blade Pr...,Hot hot hot: Razer Blade AC charger is the war...,https://www.notebookcheck.net/Hot-hot-hot-Raze...


# By the code above, we've collected the information of news.
# Next step you can do text mining for these articles!